In [17]:
import time
import smtplib
import os
import shutil
import math
import calendar
import re
import pandas as pd
import concurrent.futures
import zipfile
import shutil
import concurrent.futures
import zipfile
import shutil
import logging
import json
import requests
import tarfile
# import paramiko
# import openpyxl 
# import oracledb

from pyspark.sql import functions as F
from pyspark.sql.functions import *
from datetime import *
from pyspark.sql.window import *
from pyspark.sql import Window
from pyspark.sql.types import *
from delta import DeltaTable
from dateutil.relativedelta import relativedelta
from email.mime.text import MIMEText
from email.mime.multipart import *
from io import StringIO
from email.mime.application import MIMEApplication
from pandas import ExcelWriter
from email.mime.base import MIMEBase
from email import encoders
from tempfile import TemporaryDirectory
from datetime import *
from pytz import timezone
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, lit, when, udf
from pyspark import SparkContext
from pyspark.ml import Estimator, Model
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
# For PySpark MLlib
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.regression import LinearRegression
# from pyspark.graphframes import GraphFrame  # Graph processing library
from pyspark.streaming import StreamingContext
# from pyspark.sql.hive import HiveContext
from pyspark.ml.pipeline import Pipeline
from delta.tables import DeltaTable
from pyspark.sql.functions import pandas_udf, PandasUDFType
import unittest


# from openpyxl import Workbook
# from openpyxl.styles import PatternFill, Border, Side
# !pip install paramiko
# log_information=[]
spark = SparkSession.builder \
    .appName("HiveSupportExample") \
    .enableHiveSupport() \
    .getOrCreate()


In [2]:
import findspark
findspark.init()  

import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [10]:
def clean_string(df):
    df.createOrReplaceTempView('temp_vw')
    
    string_columns = [col_name for col_name, col_type in df.dtypes if col_type == 'string']
    
    for col_name in string_columns:
        spark.table('temp_vw') \
            .withColumn(
                col_name,
                trim(regexp_replace(col(col_name), "[\r\n\t\\xa0]", ""))
            ).createOrReplaceTempView('temp_vw')
        
        query = ", ".join(
            [f"regexp_replace({col_name}, '\\s+', ' ') as {col_name}" if c == col_name else c for c in df.columns]
        )
        sql_query = f"SELECT {query} FROM temp_vw"
        
        spark.sql(sql_query).createOrReplaceTempView('temp_vw')
    
    # Return cleaned DataFrame
    return spark.table('temp_vw')


In [7]:
data = [
    {"id": 1, "name": " Alice ", "address": "Mumbai\r\n\t", "status": "  Single\xa0"},
    {"id": 2, "name": " Bob ", "address": "Delhi\xa0\n\t", "status": "Married\n\t"},
    {"id": 3, "name": " Charlie ", "address": "Bangalore\t", "status": "  Single\r\n"},
    {"id": 4, "name": " Diana ", "address": "Chennai\t\r", "status": "Single\xa0"},
    {"id": 5, "name": " Eve ", "address": "Kolkata\r\n", "status": " Married \n"}
]

# Create DataFrame
df = spark.createDataFrame(data)

# Show the original DataFrame
print("Original DataFrame:")
df.show(truncate=False)


Original DataFrame:
+------------+---+---------+------------+
|address     |id |name     |status      |
+------------+---+---------+------------+
|Mumbai\r\n\t|1  | Alice   |  Single    |
|Delhi \n\t  |2  | Bob     |Married\n\t |
|Bangalore\t |3  | Charlie |  Single\r\n|
|Chennai\t\r |4  | Diana   |Single      |
|Kolkata\r\n |5  | Eve     | Married \n |
+------------+---+---------+------------+



In [11]:

cleaned_df = clean_string(df)
print("Cleaned DataFrame:")
cleaned_df.show(truncate=False)


Cleaned DataFrame:
+---------+---+-------+-------+
|address  |id |name   |status |
+---------+---+-------+-------+
|Mumbai   |1  |Alice  |Single |
|Delhi    |2  |Bob    |Married|
|Bangalore|3  |Charlie|Single |
|Chennai  |4  |Diana  |Single |
|Kolkata  |5  |Eve    |Married|
+---------+---+-------+-------+

